# `mistralai/Mistral-7B-v0.1`

https://huggingface.co/mistralai/Mistral-7B-v0.1


In [1]:
from huggingface_hub import login
login()

In [3]:
from transformers import pipeline
generator = pipeline('text-generation', model='mistralai/Mistral-7B-v0.1')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [4]:
generator("I want to live in India, it is", do_sample=True, min_length=20)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': 'I want to live in India, it is an incredible country. I feel like I have fallen for another place, it is happening quickly and strongly'}]

In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name="mistralai/Mistral-7B-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): MistralRMSNorm((4096,), eps=1e-0

In [8]:
model.config

MistralConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "mistralai/Mistral-7B-v0.1",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.49.0",
  "use_cache": true,
  "vocab_size": 32000
}

In [9]:
text = "I think I will go live under the"
encoded_input = tokenizer(text, return_tensors='pt')

for k in encoded_input:
    print(k)
    print(encoded_input[k].shape)
    print(encoded_input[k])


input_ids
torch.Size([1, 9])
tensor([[   1,  315, 1073,  315,  622,  576, 2943,  916,  272]])
attention_mask
torch.Size([1, 9])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])


In [10]:
model.eval()
with torch.no_grad():
    output = model(encoded_input['input_ids'])

print(output.logits.shape)
print(output.logits)

torch.Size([1, 9, 32000])
tensor([[[-5.8781, -5.8616, -0.1052,  ..., -4.3546, -3.5614, -4.0741],
         [-7.7712, -7.3810,  1.8221,  ..., -6.7385, -5.2858, -6.3371],
         [-8.3533, -8.5534,  0.0600,  ..., -7.5631, -5.7847, -5.5219],
         ...,
         [-8.9061, -9.3063,  1.9224,  ..., -8.6490, -6.4805, -8.0301],
         [-8.4487, -8.6648,  1.7403,  ..., -8.8402, -4.5676, -5.6586],
         [-7.8333, -7.8423,  0.8042,  ..., -6.2730, -5.4609, -5.1545]]])


In [11]:
logits = output.logits
probs = torch.softmax(logits,dim=-1)
out_ids = torch.argmax(probs, dim=-1)
out_ids

tensor([[  422, 28809,   315, 28809,   347,   395,   297,   264, 12997]])

In [12]:
out_text = tokenizer.decode(out_ids.squeeze(0).tolist())
out_text

'#’ I’ be with in a stairs'

In [13]:
# Another way for output seems to be 
text = "I think I will go live under the"
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
logits = output.logits
probs = torch.softmax(logits,dim=-1)
out_ids = torch.argmax(probs, dim=-1)
out_text = tokenizer.decode(out_ids.squeeze(0).tolist())
out_text


'#’ I’ be with in a stairs'

In [14]:
# shortcut generate function to generate multiple tokens

output = model.generate(**encoded_input, max_length=50)
print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


I think I will go live under the stairs.

I’m not sure what I’m going to do with my life.

I’m not sure what I’m going to do with my life.

I
